In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [2]:
import numpy as np
import torch
import scanpy as sc
import muon as mu
import anndata
import mudata
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
DATA_DIR = "/media/yob/nacho_stuff/prostate_cancer/Integration/MVAE-PoE/OtF-prostate/data/"
mdata = mu.read(DATA_DIR + "mdata_preprocessed_final_filtered.h5mu")
mdata

MuData object with n_obs × n_vars = 42475 × 31460
  obs:	'sample', 'tissue', 'patient', 'ann', 'imagerow', 'imagecol'
  2 modalities
    rna:	42475 x 18950
      obs:	'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_counts', 'n_genes', 'size_factors', 'sample', 'type', 'patient', 'ann', 'tissue', 'imagerow', 'imagecol'
      obsm:	'spatial'
      layers:	'counts', 'scran_normalization', 'spotclean'
    msi:	42475 x 12510
      obs:	'ST_barcode', 'sample', 'tissue', 'patient', 'ann', 'imagerow', 'imagecol'
      obsm:	'spatial'

In [5]:
from src.model import MVAE, MVAEParams

mvae_params = MVAEParams(beta=0.01, n_layers=2, z_dim=100, n_hidden=300)

In [6]:
torch.set_num_threads(16)
model = MVAE(mdata, mvae_params, use_cuda=True)
model

N batches for mod1:  16
N batches for mod2:  16
(42475, 18950)
(42475, 12510)


MVAE(
  (rna): ModalityLayers(
    (shared_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
    )
    (batch_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=1600, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=1600, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
    )
    (private_sampling): SamplingLayers(
      (mean): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.3, inplace=False)
      )
      (logvar): Sequential(
        (0): Linear(in_features=300, out_features=100, bias=True)
        (1): Dropout(p=0.

In [7]:
from src.train import train_mvae, TrainParams
train_params = TrainParams(train_size=0.8, n_epochs=100, learning_rate=1e-4, batch_size=128)

In [ ]:
torch.autograd.set_detect_anomaly(True)
hist = train_mvae(model, mdata, train_params)

Train data size: 33980
Test data size: 8495


 59%|████████████████████████▎                | 158/266 [12:06<08:31,  4.74s/it]